## based on Yuri's script,  find_summits.R

In [1]:
library(rtracklayer)
library(GenomicRanges)
library(plyr)
library(tidyverse)
library(data.table)

library(biosignals)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb


Attaching package: ‘plyr’


The following object is masked from ‘package:IRanges’

In [2]:
archr_dir = '/Genomics/pritykinlab/zzhao/sc-atac-submmit-calling/results/pbmc_archR_output/'
# load peaks from archR, convert the format to bed first.
all.atac.peaks <- rtracklayer::import(paste(archr_dir, "PBMC_peaks.bed", sep="/"))

In [3]:
all.atac.peaks

GRanges object with 89280 ranges and 1 metadata column:
          seqnames              ranges strand |        name
             <Rle>           <IRanges>  <Rle> | <character>
      [1]     chr1       817097-817596      * |         501
      [2]     chr1       820722-821221      * |         501
      [3]     chr1       826609-827108      * |         501
      [4]     chr1       827291-827790      * |         501
      [5]     chr1       857896-858395      * |         501
      ...      ...                 ...    ... .         ...
  [89276]     chrX 155611289-155611788      * |         501
  [89277]     chrX 155612712-155613211      * |         501
  [89278]     chrX 155632372-155632871      * |         501
  [89279]     chrX 155841284-155841783      * |         501
  [89280]     chrX 155881050-155881549      * |         501
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [4]:
names(all.atac.peaks) <- sapply(1:length(all.atac.peaks), function(i) paste("peak", i, sep=""))

In [5]:
all.atac.peaks

GRanges object with 89280 ranges and 1 metadata column:
            seqnames              ranges strand |        name
               <Rle>           <IRanges>  <Rle> | <character>
      peak1     chr1       817097-817596      * |         501
      peak2     chr1       820722-821221      * |         501
      peak3     chr1       826609-827108      * |         501
      peak4     chr1       827291-827790      * |         501
      peak5     chr1       857896-858395      * |         501
        ...      ...                 ...    ... .         ...
  peak89276     chrX 155611289-155611788      * |         501
  peak89277     chrX 155612712-155613211      * |         501
  peak89278     chrX 155632372-155632871      * |         501
  peak89279     chrX 155841284-155841783      * |         501
  peak89280     chrX 155881050-155881549      * |         501
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [6]:
# load bam coverage file (big wig format)
atac.signal <- rtracklayer::import("sc_atac.bw", as = "RleList")

In [7]:
head(atac.signal)

RleList of length 6
$chr1
numeric-Rle of length 248956422 with 13515643 runs
  Lengths:       9870          4          1 ...          1       9889
  Values :   0.000000   0.111950   0.139938 ...  0.0279876  0.0000000

$chr10
numeric-Rle of length 133797422 with 6297921 runs
  Lengths:       9883          4          1 ...          6       9891
  Values :  0.0000000  0.0279876  0.0559751 ...  0.0279876  0.0000000

$chr11
numeric-Rle of length 135086622 with 6862840 runs
  Lengths:     61286       245         9 ...         1         5      9877
  Values : 0.0000000 0.0559751 0.1119500 ... 0.1399380 0.0559751 0.0000000

$chr12
numeric-Rle of length 133275309 with 7001202 runs
  Lengths:       9883          6         51 ...         22       9866
  Values :  0.0000000  0.0279876  0.0559751 ...  0.0279876  0.0000000

$chr13
numeric-Rle of length 114364328 with 3156346 runs
  Lengths:  16002120        43       420 ...         8       106      9871
  Values : 0.0000000 0.0559751 0.0000000 ... 0

In [8]:
# subsetting the bam coverage by archR picked peaks
peaks.signal <- atac.signal[all.atac.peaks]

In [9]:
names(peaks.signal) <- names(all.atac.peaks)

In [11]:
peaks.signal$peak1

numeric-Rle of length 500 with 333 runs
  Lengths:        2        5        3        1 ...        1        1        1
  Values :  7.58463  7.52865  7.64060  7.75255 ...  9.93558  9.76766  9.73967

In [16]:
# summit calling using biosignals package from Leslie Lab: https://bitbucket.org/leslielab/biosignals
findVectorMax <- function(v, g0, g1, g2, min.dist = 100) {
    v.conv <- convolve1d(as.numeric(v), g0)
    v.conv.d1 <- convolve1d(as.numeric(v), g1)
    v.conv.d2 <- convolve1d(as.numeric(v), g2)
    v.conv.d1.zero <- zeroCrossings(v.conv.d1)
    points.max <- v.conv.d1.zero[v.conv.d2[v.conv.d1.zero] < 0]
    if (length(points.max) == 0) {
        return(c())
    }
    points.max <- data.table(x = points.max)
    points.max$value <- v.conv[points.max$x]
    points.max <- points.max[order(-value)]
    vector.max <- c()
    for (i in 1:nrow(points.max)) {
        x <- points.max[i, ][, x]
        if (min(abs(x - c(vector.max, 0, length(v)))) >= min.dist) {
            vector.max <- c(vector.max, x)
        }
    }
    vector.max
}


In [17]:
gen_kernels <- function(bw) {
    g0 <- generateKernel("gaussian", bandwidth = bw, deriv = 0)
    g1 <- generateKernel("gaussian", bandwidth = bw, deriv = 1)
    g2 <- generateKernel("gaussian", bandwidth = bw, deriv = 2)
    list("g0"=g0, "g1"=g1, "g2"=g2)
}

In [18]:
kernels <- gen_kernels(150)
peaks.summits <- sapply(peaks.signal, function(peak) findVectorMax(peak, kernels$g0, kernels$g1, kernels$g2))

In [19]:
summit.count <- sapply(peaks.summits, length)
peaks.s0.signal <- peaks.signal[summit.count == 0]
length(peaks.s0.signal)

[1] 15926

In [20]:
length(peaks.signal[summit.count > 0])

[1] 73354

In [21]:
max(summit.count)

[1] 3

In [22]:
print("here")

[1] "here"


## now use smaller min distance, same kernel bandwidth

In [26]:
peaks.s0.summits <- sapply(peaks.s0.signal, findVectorMax, g0 = kernels$g0, g1 = kernels$g1, g2 = kernels$g2,
                           min.dist = 50)

summit.s0.count <- sapply(peaks.s0.summits, length)
peaks.s0.s0.signal <- peaks.s0.signal[summit.s0.count == 0]
peaks.s0.s0.signal.max <- sapply(peaks.s0.s0.signal, max)
peaks.s0.s0.signal.high <- peaks.s0.s0.signal[peaks.s0.s0.signal.max > 0.5]

In [27]:
print(length(peaks.s0.s0.signal))

[1] 13988


In [28]:
print(sum(summit.s0.count > 0))

[1] 1938


In [29]:
print(sum(summit.s0.count == 0))

[1] 13988


In [30]:
length(peaks.s0.s0.signal.high)

[1] 13986

## now use smaller bandwidth 

In [31]:
kernels_100bw <- gen_kernels(100)
peaks.s0.s0.high.summits <- sapply(peaks.s0.s0.signal.high, findVectorMax, g0 = kernels_100bw$g0, 
                                   g1 = kernels_100bw$g1, g2 = kernels_100bw$g2)


In [32]:
summit.s0.s0.high.count <- sapply(peaks.s0.s0.high.summits, length)


In [33]:
sum(summit.s0.s0.high.count > 0)

[1] 2726

In [34]:
sum(summit.s0.s0.high.count == 0)

[1] 11260

In [35]:
peaks.summits.final <- c(peaks.summits[summit.count > 0],
                         peaks.s0.summits[summit.s0.count > 0],
                         peaks.s0.s0.high.summits[summit.s0.s0.high.count > 0])

In [36]:
length(peaks.summits.final)

[1] 78018

In [37]:
head(peaks.summits.final)

$peak1
[1] 294

$peak2
[1] 240

$peak4
[1] 364

$peak5
[1] 249

$peak6
[1] 210

$peak7
[1] 280

In [38]:
tail(peaks.summits.final)

$peak89141
[1] 314

$peak89147
[1] 400

$peak89148
[1] 145 252 400

$peak89163
[1] 298

$peak89167
[1] 323

$peak89176
[1] 298

In [39]:
length(peaks.signal)

[1] 89280

In [40]:
peaks.signal$peak2

numeric-Rle of length 500 with 167 runs
  Lengths:      16       1       4       4 ...       7       1       4       7
  Values : 2.49089 2.37894 2.49089 2.43492 ... 3.13461 3.07863 3.24656 3.27454

In [41]:
peaks.signal$peak2[448]

numeric-Rle of length 1 with 1 run
  Lengths:       1
  Values : 3.02266

In [42]:
max(peaks.signal$peak2)

[1] 3.6104

In [43]:
length(peaks.signal$peak2)

[1] 500

In [44]:
saveRDS(peaks.summits.final, "all-atac-peaks-summits-lists.rds")

# generate final set of regions around peak summits

In [45]:
tail(peaks.summits.final)

$peak89141
[1] 314

$peak89147
[1] 400

$peak89148
[1] 145 252 400

$peak89163
[1] 298

$peak89167
[1] 323

$peak89176
[1] 298

In [46]:
print("prepare final set of regions around peak summits")
summit.width <- 150
peaks.summits.dt <- data.table(ldply(peaks.summits.final, matrix, .id = "peak"))
colnames(peaks.summits.dt) <- c("peak", "summit")
peaks.summits <- all.atac.peaks[peaks.summits.dt$peak, ]

[1] "prepare final set of regions around peak summits"


In [47]:
stopifnot(start(peaks.summits) + peaks.summits.dt$summit <= end(peaks.summits))
start(peaks.summits) <- start(peaks.summits) + peaks.summits.dt$summit
end(peaks.summits) <- start(peaks.summits)

In [48]:
peaks.summits$peak <- names(peaks.summits)
peaks.summits

GRanges object with 79036 ranges and 2 metadata columns:
            seqnames    ranges strand |        name        peak
               <Rle> <IRanges>  <Rle> | <character> <character>
      peak1     chr1    817391      * |         501       peak1
      peak2     chr1    820962      * |         501       peak2
      peak4     chr1    827655      * |         501       peak4
      peak5     chr1    858145      * |         501       peak5
      peak6     chr1    865811      * |         501       peak6
        ...      ...       ...    ... .         ...         ...
  peak89148     chrX 153941743      * |         501   peak89148
  peak89148     chrX 153941891      * |         501   peak89148
  peak89163     chrX 153982739      * |         501   peak89163
  peak89167     chrX 154002379      * |         501   peak89167
  peak89176     chrX 154096595      * |         501   peak89176
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [49]:
peaks.summits$peak_position <- sapply(peaks.summits$peak, function (pk) {
    peak_info = all.atac.peaks[pk]
    paste(seqnames(peak_info), ":", start(ranges(peak_info)), "-", end(ranges(peak_info)), sep = "")
})
peaks.summits

GRanges object with 79036 ranges and 3 metadata columns:
            seqnames    ranges strand |        name        peak
               <Rle> <IRanges>  <Rle> | <character> <character>
      peak1     chr1    817391      * |         501       peak1
      peak2     chr1    820962      * |         501       peak2
      peak4     chr1    827655      * |         501       peak4
      peak5     chr1    858145      * |         501       peak5
      peak6     chr1    865811      * |         501       peak6
        ...      ...       ...    ... .         ...         ...
  peak89148     chrX 153941743      * |         501   peak89148
  peak89148     chrX 153941891      * |         501   peak89148
  peak89163     chrX 153982739      * |         501   peak89163
  peak89167     chrX 154002379      * |         501   peak89167
  peak89176     chrX 154096595      * |         501   peak89176
                     peak_position
                       <character>
      peak1     chr1:817097-817596
      

In [50]:
peaks.summits$peak <- peaks.summits$name

In [51]:
peaks.summits <- sort(peaks.summits)
peaks.summits$name <- paste0("summit", seq_along(peaks.summits))

In [52]:
peaks.summits

GRanges object with 79036 ranges and 3 metadata columns:
            seqnames    ranges strand |        name        peak
               <Rle> <IRanges>  <Rle> | <character> <character>
      peak1     chr1    817391      * |     summit1         501
      peak2     chr1    820962      * |     summit2         501
      peak4     chr1    827655      * |     summit3         501
      peak5     chr1    858145      * |     summit4         501
      peak6     chr1    865811      * |     summit5         501
        ...      ...       ...    ... .         ...         ...
  peak89276     chrX 155611564      * | summit79032         501
  peak89277     chrX 155612866      * | summit79033         501
  peak89278     chrX 155632683      * | summit79034         501
  peak89279     chrX 155841526      * | summit79035         501
  peak89280     chrX 155881294      * | summit79036         501
                     peak_position
                       <character>
      peak1     chr1:817097-817596
      

In [55]:
names(peaks.summits) <- peaks.summits$name
peaks.summits <- resize(peaks.summits, width = summit.width, fix = "center")

In [56]:
peaks.summits

GRanges object with 79036 ranges and 3 metadata columns:
              seqnames              ranges strand |        name        peak
                 <Rle>           <IRanges>  <Rle> | <character> <character>
      summit1     chr1       817316-817465      * |     summit1         501
      summit2     chr1       820887-821036      * |     summit2         501
      summit3     chr1       827580-827729      * |     summit3         501
      summit4     chr1       858070-858219      * |     summit4         501
      summit5     chr1       865736-865885      * |     summit5         501
          ...      ...                 ...    ... .         ...         ...
  summit79032     chrX 155611489-155611638      * | summit79032         501
  summit79033     chrX 155612791-155612940      * | summit79033         501
  summit79034     chrX 155632608-155632757      * | summit79034         501
  summit79035     chrX 155841451-155841600      * | summit79035         501
  summit79036     chrX 15588121

In [57]:
export(peaks.summits, "all-atac-summits.bed")
saveRDS(peaks.summits, "all-atac-summits.rds")

In [58]:
write.table(as.data.frame(peaks.summits), "peak_summits.more_info.bed", quote= F, row.names = F, sep ="\t")